In [7]:
import pandas as pd
from sklearn.metrics import confusion_matrix

MODEL_NAME = "LSOIE++"


def get_dev_result(model_name: str):
    model_path = "..\evaluations\\" + model_name + "_dev.txt"
    with open(model_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        lines = [line.strip().split(" ") for line in lines]
        y_true = [line[1] for line in lines if len(line) == 3]
        y_pred = [line[2] for line in lines if len(line) == 3]
        return y_true, y_pred
y_true, y_pred = get_dev_result(MODEL_NAME)

def get_confusion_matrix(y_true, y_pred):
    labels = list(set(y_true))
    df = pd.DataFrame(
        data=confusion_matrix(y_true, y_pred, labels=labels),
        columns=labels,
        index=labels,
    )
    df.index.name = "Y-True"
    df.columns.name = "Y-Pred"
    return df

#
# Local (metrics per class)
#
df = get_confusion_matrix(y_true, y_pred)

labels = list(set(y_true))
tps = {}
fps = {}
fns = {}
for label in labels:
    tps[label] = df.loc[label, label]
    fps[label] = df[label].sum() - tps[label]
    fns[label] = df.loc[label].sum() - tps[label]

#
# Global
#
micro_averages = {}
macro_averages = {}

correct_predictions = sum(tps.values())

total_predictions = df.values.sum()
accuracy_global = round(correct_predictions / total_predictions,4 ) if total_predictions > 0. else 0.

print("#-- Local measures --#")
print("True Positives:", tps)
print("False Positives:", fps)
print("False Negatives:", fns)

print("\n#-- Global measures --#")
print("Correct predictions:", correct_predictions)
print("Total predictions:", total_predictions)
print("Accuracy:", accuracy_global)

#-- Local measures --#
True Positives: {'I-V': 19, 'O': 603, 'B-V': 33, 'I-ARG1': 24, 'B-ARG0': 35, 'I-ARG0': 55, 'B-ARG1': 18}
False Positives: {'I-V': 8, 'O': 150, 'B-V': 7, 'I-ARG1': 46, 'B-ARG0': 7, 'I-ARG0': 7, 'B-ARG1': 17}
False Negatives: {'I-V': 50, 'O': 64, 'B-V': 18, 'I-ARG1': 32, 'B-ARG0': 16, 'I-ARG0': 29, 'B-ARG1': 33}

#-- Global measures --#
Correct predictions: 787
Total predictions: 1029
Accuracy: 0.7648


In [8]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

      B-ARG0     0.8333    0.6863    0.7527        51
      B-ARG1     0.5143    0.3529    0.4186        51
         B-V     0.8250    0.6471    0.7253        51
      I-ARG0     0.8871    0.6548    0.7534        84
      I-ARG1     0.3429    0.4286    0.3810        56
         I-V     0.7037    0.2754    0.3958        69
           O     0.8008    0.9040    0.8493       667

    accuracy                         0.7648      1029
   macro avg     0.7010    0.5641    0.6109      1029
weighted avg     0.7650    0.7648    0.7533      1029



In [9]:
df.style.background_gradient(cmap="Purples")

Y-Pred,I-V,O,B-V,I-ARG1,B-ARG0,I-ARG0,B-ARG1
Y-True,,,,,,,
I-V,19,33,1,8,0,0,8
O,7,603,5,32,4,7,9
B-V,0,18,33,0,0,0,0
I-ARG1,0,32,0,24,0,0,0
B-ARG0,0,16,0,0,35,0,0
I-ARG0,0,25,1,0,3,55,0
B-ARG1,1,26,0,6,0,0,18


In [27]:
from transformers import MarianMTModel, MarianTokenizer, pipeline

pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")
sents = ["The world is very big", "I am a student", "I am a teacher", "I live in Brazil"]
model_name = "Helsinki-NLP/opus-mt-tc-big-en-pt"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
sent = "The Olympic Games take place every four years."
print(pipe(sent))


KeyboardInterrupt

